In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate

In [2]:
train_df = pd.read_csv('/Users/sola/Downloads/open/train.csv')
test_df = pd.read_csv('/Users/sola/Downloads/open/test.csv')

In [3]:
# 결측치에 평균 넣어주기
train_df=train_df.fillna(train_df.mean())
test_df=test_df.fillna(test_df.mean())

/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_10134/1987006743.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_df=train_df.fillna(train_df.mean())
/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_10134/1987006743.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_df=test_df.fillna(test_df.mean())


In [13]:
train_df['X_2875']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
593    0.0
594    0.0
595    0.0
596    0.0
597    0.0
Name: X_2875, Length: 598, dtype: float64

In [14]:
# 모든 값이 0인 피처 지울거임..
wrong = []
for i in range(1,2876):
    ch = 'X_'+str(i)
    if train_df[ch].mean() == 0:
        wrong.append(ch)

print(wrong)

['X_3', 'X_6', 'X_14', 'X_70', 'X_74', 'X_75', 'X_76', 'X_77', 'X_78', 'X_79', 'X_82', 'X_83', 'X_84', 'X_85', 'X_142', 'X_190', 'X_191', 'X_192', 'X_193', 'X_194', 'X_195', 'X_232', 'X_234', 'X_235', 'X_259', 'X_260', 'X_315', 'X_319', 'X_322', 'X_323', 'X_324', 'X_327', 'X_328', 'X_329', 'X_330', 'X_375', 'X_376', 'X_377', 'X_378', 'X_427', 'X_466', 'X_467', 'X_505', 'X_508', 'X_558', 'X_561', 'X_567', 'X_583', 'X_617', 'X_620', 'X_631', 'X_633', 'X_634', 'X_636', 'X_638', 'X_639', 'X_640', 'X_641', 'X_642', 'X_672', 'X_673', 'X_676', 'X_691', 'X_692', 'X_695', 'X_715', 'X_729', 'X_732', 'X_743', 'X_749', 'X_759', 'X_760', 'X_761', 'X_764', 'X_776', 'X_777', 'X_778', 'X_836', 'X_843', 'X_844', 'X_849', 'X_859', 'X_886', 'X_887', 'X_888', 'X_889', 'X_934', 'X_935', 'X_936', 'X_937', 'X_992', 'X_1020', 'X_1021', 'X_1022', 'X_1023', 'X_1024', 'X_1025', 'X_1070', 'X_1092', 'X_1119', 'X_1137', 'X_1146', 'X_1206', 'X_1216', 'X_1219', 'X_1248', 'X_1249', 'X_1250', 'X_1251', 'X_1252', 'X_125

In [15]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [16]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_x = train_x.drop(columns=wrong) # 평균이 0인 컬럼은 지움
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP']) # 평균이 0인 컬럼은 지움
test_x = test_x.drop(columns=wrong)

In [17]:
# 범주형을 수치형으로 변환하는 과정
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])
print('Done.')

Done.


In [18]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_4,X_5,X_7,X_8,X_9,X_10,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,2,0,2.409742,95.123209,45.0,10.39255,48.802292,10.048711,41.469914,2.0,...,189.000000,383.000000,368.296296,353.000000,39.3400,40.8900,32.5600,34.0900,77.7700,1.0
1,3,0,2.409742,95.123209,45.0,10.39255,48.802292,10.048711,41.469914,2.0,...,185.600000,383.000000,367.735849,353.000000,38.8900,42.8200,43.9200,35.3400,72.5500,1.0
2,2,0,2.409742,95.123209,45.0,10.39255,48.802292,10.048711,41.469914,2.0,...,165.500000,383.000000,367.320755,353.000000,39.1900,36.6500,42.4700,36.5300,78.3500,1.0
3,3,0,2.409742,95.123209,45.0,10.39255,48.802292,10.048711,41.469914,2.0,...,165.800000,384.000000,369.188679,353.000000,37.7400,39.1700,52.1700,30.5800,71.7800,1.0
4,2,0,2.409742,95.123209,45.0,10.39255,48.802292,10.048711,41.469914,2.0,...,182.600000,383.000000,367.351852,352.000000,38.7000,41.8900,46.9300,33.0900,76.9700,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,2.000000,95.000000,45.0,10.00000,50.000000,10.000000,52.000000,2.0,...,163.290763,423.558233,406.088187,388.064257,50.8073,53.6077,49.6062,51.6598,66.6497,1.0
594,2,0,2.409742,95.123209,45.0,10.39255,48.802292,10.048711,41.469914,2.0,...,168.700000,384.000000,369.811321,353.000000,49.4700,53.0700,50.8900,55.1000,66.4900,1.0
595,2,0,2.409742,95.123209,45.0,10.39255,48.802292,10.048711,41.469914,2.0,...,156.600000,383.000000,367.018868,352.000000,50.8073,53.6077,49.6062,51.6598,66.6497,1.0
596,4,1,40.000000,94.000000,45.0,11.00000,45.000000,10.000000,31.000000,2.0,...,163.290763,423.558233,406.088187,388.064257,50.8073,53.6077,49.6062,51.6598,66.6497,1.0


In [19]:
hgb = HistGradientBoostingClassifier(random_state=37)

# 교차검증
scores = cross_validate(hgb, train_x, train_y, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.6188375350140056


In [20]:
from sklearn.inspection import permutation_importance

# 학습시키기
hgb.fit(train_x, train_y)
#특성중요도 확인
# result = permutation_importance(hgb, train_x, train_y, n_repeats=10,random_state=37, n_jobs=-1)
# print(result.importances_mean)

HistGradientBoostingClassifier(random_state=37)

In [21]:
# hgb.score(test_input, test_target)

In [22]:
preds = hgb.predict(test_x)

In [23]:
preds

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [24]:
submit = pd.read_csv('/Users/sola/Downloads/open/sample_submission.csv')

In [25]:
submit['Y_Class'] = preds

In [26]:
submit.to_csv('/Users/sola/Downloads/open/baseline_submission6.csv', index=False)

히스토그램 기반 그레이디언트 부스팅 : submission3
-> 결측치에 전부 0 입력
정확도 0.541749409

히스토그램 기반 그레이디언트 부스팅 : submission4
-> 결측치에 컬럼별 평균치넣고 아무것도 없는 컬럼에는 0 입력
정확도

히스토그램 기반 그레이디언트 부스팅 : submission6
-> 결측치에 컬럼별 평균치넣고 아무것도 없는 컬럼에는 0 입력 + 평균이 0인 컬럼은 전부 제외함([wrong])
정확도 0.4498072733 개형편없넹..!